# Preprocessing of ArcticDEM, CryoSat-2 and Sentinel-1

This script does all of the preprocessing of the elevation and Sentinel-1 data needed before training and applaying UDEM.

Before you start running this script you need to download the following files from the original data sources:
- The ArcticDEM mosaic (I have used the 100m version)
- RGI7 of the region (RGI2000-v7.0-C-03_arctic_canada_north)
- CryoSat2 swath measurements of your region (DOI to be added).


My region of interest is the Northern Canadian Arctic, but for computational purposes I have sub-divided that region into 6 smaller ones.

However, you can in principle do this for any region of interest as long as you have 
- A DEM and DEM strips (ArcticDEM or REMA for instance)
- CryoSat-2 swath measurements
- Sentinel-1 coverage
- A shapefile (or any other geodata file which geopandas can read) of the region of interest


Note! I have only tested the scripts on Mac (M4 Pro). I believe that they should work on all UNIX systems as well, but to work on Windows you need to adjust paths "/" to "\\".

In [ ]:
import geopandas
import os
import preprocessing_tools as ptools
import ee

In [ ]:
# Define the project directory
projectDir = "/Users/rfk471/Dropbox/elevation-canada"

project_crs = "EPSG:3413"

region_ids = ["01","02","03","04","05","06"]

## 0. Prepare ArcticDEM mosaic for the desire region

Clip, reproject and save the ArcticDEM mosaic of the desired region to reduce computations later on.

This part requires GDAL installed.

In [ ]:
# Full path to the ArcticDEM mosaic
demDir = f"{projectDir}/data/initial/ArcticDEM/arcticdem_mosaic_100m_v4.1_dem.tif"

for region_id in region_ids:
    region = geopandas.read_file(f"{projectDir}/data/initial/regions/region-{region_id}.shp")
    ptools.clip_reproject_arcticdem(region,region_id,demDir,projectDir)

## 1. Save seasonal CryoSat-2 tifs

This part loads the CryoSat-2 swath files, cleans the data by excluding data points that differ more than 25 m from a reference DEM, and saves seasonal elevations by taking the median of the monthly elevations.

The seasons are defined like this:
Winter: Dec, Jan, Feb
Spring: Mar, Apr, May
Summer: Jun, Jul, Aug
Fall: Sep, Oct, Nov

The CryoSat-2 swath data used here has been downloaded from https://doi.org/10.4121/955d7f5a-0e3f-4166-a411-f0dcc4557cb2

You can also do your own preprocessig of CryoSat-2 by using packages such as cryoswath (which was used for the data downloaded here)

In [ ]:
# Paths
dataDir = f"{projectDir}/data/initial/cryoswath/"

for region_id in region_ids:

    # Pick the right dataset depending on region
    if region_id == "01":
        CSfile = "Glacier_surface_elevation__Arctic_Canada_North__Axel_Heiberg_and_Meighen_Is__monthly_500x500m.nc"
    elif region_id == "02":
        CSfile = "Glacier_surface_elevation__Arctic_Canada_North__North_Ellesmere_Island__monthly_500x500m.nc"
    elif region_id == "03":
        CSfile = "Glacier_surface_elevation__Arctic_Canada_North__North_Central_Ellesmere_Island__monthly_500x500m.nc"
    elif region_id == "04":
        CSfile = "Glacier_surface_elevation__Arctic_Canada_North__South_Central_Ellesmere_Island__monthly_500x500m.nc"
    elif region_id == "05":
        CSfile = "Glacier_surface_elevation__Arctic_Canada_North__South_Ellesmere_Island-Northwest_Devon__monthly_500x500m.nc"
    elif region_id == "06":
        CSfile = "Glacier_surface_elevation__Arctic_Canada_North__Devon_Island__monthly_500x500m.nc"


    ptools.save_seasonal_cryosat(region_id,dataDir,CSfile,projectDir)


## 2. Download Sentinel-1

This part requires a Google Earth Engine profile, and the Sentinel-1 files are downloaded to your Google Drive. 

The script computes and saves the seasonal means.

!!! Be aware that if you, like me, don't have a lot of storage space in your Google Drive, you might need to start downloading the images before they have all been processed to not run into storage issues !!!

In [ ]:
# Run the below line if you haven't already authenticated your ee account

#ee.Authenticate()

In [ ]:
# Initiliaze your ee-project, remember to change the name to the name of your project
ee.Initialize(project='bedcan')

# Downloading everything might take some time... Be patient... Or at least try...
for region_id in region_ids:
    ptools.download_sentinel1(region_id,projectDir)

## 3. Downloading ArcticDEM strips

This part takes quite a long time and has been put into a separate script.

It downloads the 2m strips of a desired region (only the part of the region which intersect with the RGI shapefile of the region) in parallel, interpolates it onto a 50 m grid, and removes the original 2 m strips to save space.

It requires the following data:
- A shapefile of the region of interest
- A shapefile of all strip indecies (e.g., ArcticDEM_Strip_Index_s2s041.shp, can be downloaded from https://www.pgc.umn.edu/data/arcticdem/)
- The RGI v7 shapefile (to reduce number of strips to be downloaded)

I recommend to run the script directly from the terminal like this:

`python download-arcticdem.py --region_id 02 --project_crs EPSG:3413 --n_threads 8 --projectDir "/Users/rfk471/Dropbox/elevation-canada"`

Or like this to prevent your laptop (mac at least) to sleep:

`caffeinate -i python download-arcticdem.py --region_id 02 --project_crs EPSG:3413 --n_threads 8 --projectDir "/Users/rfk471/Dropbox/elevation-canada"`



`--region_id`: region_id as here, but you can also modify the script to your liking

`--project_crs`: crs of the project, here we use Arctic Polar Stereographic

`--n_threads`: Number of parallel threads to use (I have used 8 on a laptop with 14 available)

`--projectDir`: The path to the projectDir just like defined here

## 4. Create seasonal ArcticDEM tifs

This part requires a few steps:
1. Bit-masking: Mask out all non-good pixels based on the bit mask provided.
2. Co-register the strips to the actual surface, by fitting a line through the residuals of the strip elevations and some reference elevation.
3. Create seasonal means from the individual strips.


Reference elevation:

For the reference elevation I use the corresponding seasonal CryoSat-2 geotif in ice-covered areas and the ArcticDEM mosaic for ice-free areas. I use RGI v7 to determine whether a pixel is ice-covered or ice-free.

In [ ]:
# Bit masking

n_threads = 8 # Number of CPU cores to use

for region_id in region_ids:
    ptools.bit_masking(region_id,projectDir,n_threads)

In [ ]:
# Co-registration

n_threads = 2

for region_id in region_ids:
    ptools.co_registration(region_id,project_crs,projectDir,n_threads)

In [ ]:
# Seasonal means

for region_id in region_ids:
    ptools.seasonal_arcticdem(region_id,projectDir)

## 5. Regrid Sentinel-1

Here, the Sentinel-1 files are regridded onto the same "common" grid as CryoSat-2 and ArcticDEM.

Be aware that the Sentinel-1 files are assumed to be located here: `f"{projectDir}/data/initial/sentinel-1/region-{region_id}/"`

In [ ]:

for region_id in region_ids:
    ptools.regrid_sentinel1(region_id,projectDir)

## 6. Create icemask

Create and save icemask based on the region and RGI v7

In [ ]:
for region_id in region_ids:
    ptools.get_icemask(region_id, projectDir, project_crs)

## 7. Create tile file

This part creates a geopackage which defines the tiles used for the U-Net. The tiles are based on the 100 m mask. The script only generates tiles of ice-covered areas. The tiles are 64x64 pixels and have an 8 pixel overlap in all directions.

In [ ]:
for region_id in region_ids:
    ptools.create_tiles(region_id, projectDir)

## 8. Create netcdfs

The point of this part is to generate 3 netcdf files with each of the three datasets as they are used in the U-Net model. 

All three datasets go through the following steps:
1. Fill NaN pixels in a nearest neighbour manner in a maximum distance of 5 pixels of a data-filled pixel.
2. (CryoSat-2 only) interpolate data onto the high-resolution grid (same grid as S-1 and ArcticDEM).
3. Apply the icemask to mask out pixels in ice-free areas and set them to 0, and treat 0 as a NO DATA value.
4. Concatenate all dataframes along a time dimension and save to netcdf.

In [ ]:

for region_id in region_ids:
    ptools.create_netcdfs(region_id,projectDir)

## 9. Normalize, tile, and split data

Nomalization: You have the choice to normalize the data or not. The valid options are `"raw"` (no normalization) and `"zscore"` (all data is normalized in a zscore manner).

Tiling: All data is clipped into smaller tiles using the predefined tiles. The tiled data is saved in a hdf5 file with the following structure:

```
seasonal_tiles_normalization_{normalization}.h5
├── 2014_10
│   ├── tile_001
│   │   ├── cs_tile
│   │   ├── s1_tile
│   │   ├── adem_tile
│   │   ├── mask_tile
│   │   └── (attrs: cs_flag, s1_flag, adem_flag)
│   ├── tile_002
│   │   ├── ...
│   └── ...
├── 2015_01
│   └── ...
└── ...
```

Data splitting:

All tiles where S1 and CryoSat2 have full coverage are put into the `all_tiles` dataframe and saved as `X_all.pkl`. This is the data which we eventually want to predict on once we have a working U-Net.

All tiles where all three datasets have full coverage are put into the `complete_tiles` dataframe. This data is further split into training data (80%), test data (10%) and validation data (10%). HPO will be performed on the validation dataset on which we do a 80/20 split.


In [ ]:
normalization = "raw"
for region_id in region_ids:
    ptools.normalize_tiling(region_id,projectDir,project_crs,normalization)

In [ ]:
complete_tiles, all_tiles = ptools.load_complete_tiles(region_ids,normalization,projectDir)

random_number = 7

ptools.split_save_data(complete_tiles,all_tiles,projectDir,normalization,random_number)

In [ ]:
print(f"There are in total {len(complete_tiles)} tiles with full coverage of all datasets.")